In [15]:
import ee
import pandas as pd
import json

# # Trigger the authentication flow.
# ee.Authenticate()

# Initialize the library.
ee.Initialize()

In [16]:
def import_aois(csv_loc):    

    df_labels = pd.read_csv(csv_loc)
    df_labels = df_labels[["center-lat","center-long","polygon","Labels combined"]]

    polygons = []
    for polygon in df_labels["polygon"]:
        polygons.append(json.loads(polygon)["coordinates"])

    return polygons

### CHANGE BELOW PATH ###
zhenya_path = "/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Labelled/Tiles_v3/Polygon_List/polygons_101320.csv"
david_path = 'D:/canopy_data/csvs/polygons_101320.csv'
polygons = import_aois(zhenya_path)

In [17]:
def get_s2_sr_cld_col(aoi, start_date, end_date):
    # Import and filter S2 SR.
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR')
        .filterBounds(aoi)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))

    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi)
        .filterDate(start_date, end_date))

    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))

In [18]:
def add_cloud_bands(img):
    # Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

    # Condition s2cloudless by the probability threshold value.
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')

    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb, is_cloud]))

In [19]:
def add_shadow_bands(img):
    # Identify water pixels from the SCL band.
    not_water = img.select('SCL').neq(6)

    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')

    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));

    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')

    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))

In [20]:
def add_cld_shdw_mask(img):
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)

    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)

    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focal_min(2).focal_max(BUFFER*2/20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
        .rename('cloudmask'))

    # Add the final cloud-shadow mask to the image.
    return img_cloud_shadow.addBands(is_cld_shdw)

In [29]:
def apply_cld_shdw_mask_all_bands(img):
    # Subset the cloudmask band and invert it so clouds/shadow are 0, else 1.
    not_cld_shdw = img.select('cloudmask').Not()

    # Subset reflectance bands and update their masks, return the result.
    return img.updateMask(not_cld_shdw)

In [24]:
AOI = ee.Geometry.Polygon(polygons[0])
START_DATE = '2019-01-01'
END_DATE = '2020-12-31'
CLOUD_FILTER = 60
CLD_PRB_THRESH = 40
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST = 2
BUFFER = 100

In [25]:
s2_sr_cld_col = get_s2_sr_cld_col(AOI, START_DATE, END_DATE)

In [27]:
s2_sr_median = (s2_sr_cld_col.map(add_cld_shdw_mask)
                             .map(apply_cld_shdw_mask_all_bands)
                             .median())

In [63]:
def export_to_gcs(s2_sr_median,AOI,polygon_id,date_range):
    export = ee.batch.Export.image.toCloudStorage(
      image=s2_sr_median.select(['TCI_R', 'TCI_G', 'TCI_B']),
      description="s2cloudless_misha_polygon_export",
      scale=10,
      region=AOI,
      fileNamePrefix=f'S2_CloudFree/s2cloudless_misha_polygons/{date_range + "_" + str(polygon_id)}',
      bucket='project-canopy-temp-2',
      maxPixels=1e13
)
    export.start()
    
    return export

In [33]:
export = export_to_gcs()

In [34]:
while export.active():
    print(export.status(), end="\r", flush=True)

In [42]:
dates_dict = {"3_months":{"START_DATE":'2019-12-11',"END_DATE":'2020-03-10'},
"2_years":{"START_DATE":"2019-01-01","END_DATE":"2020-12-31"}}

In [64]:
def s2cloudless_download(dates_dict,polygons):
    for date_range in dates_dict.keys():
        for polygon_id,polygon in enumerate(polygons,1):
            print(f"processing {polygon_id} of {len(polygons)} for {date_range} range")
            AOI = ee.Geometry.Polygon(polygon)
            s2_sr_cld_col = get_s2_sr_cld_col(AOI, dates_dict[date_range]["START_DATE"], dates_dict[date_range]["END_DATE"])
            s2_sr_median = (s2_sr_cld_col.map(add_cld_shdw_mask)
                             .map(apply_cld_shdw_mask_all_bands)
                             .median())
            export_to_gcs(s2_sr_median,AOI,polygon_id,date_range)
            
        
    

In [65]:
s2cloudless_download(dates_dict,polygons)

processing 1 of 101 for 3_months range
processing 2 of 101 for 3_months range
processing 3 of 101 for 3_months range
processing 4 of 101 for 3_months range
processing 5 of 101 for 3_months range
processing 6 of 101 for 3_months range
processing 7 of 101 for 3_months range
processing 8 of 101 for 3_months range
processing 9 of 101 for 3_months range
processing 10 of 101 for 3_months range
processing 11 of 101 for 3_months range
processing 12 of 101 for 3_months range
processing 13 of 101 for 3_months range
processing 14 of 101 for 3_months range
processing 15 of 101 for 3_months range
processing 16 of 101 for 3_months range
processing 17 of 101 for 3_months range
processing 18 of 101 for 3_months range
processing 19 of 101 for 3_months range
processing 20 of 101 for 3_months range
processing 21 of 101 for 3_months range
processing 22 of 101 for 3_months range
processing 23 of 101 for 3_months range
processing 24 of 101 for 3_months range
processing 25 of 101 for 3_months range
processin